In [1]:
import cv2 as cv

In [2]:
import pytesseract
print(pytesseract.get_tesseract_version())

5.5.0.20241111


In [3]:
from PIL import Image

# Cargar la imagen
image_path = "temp/Ejemplo.png"
image = Image.open(image_path)

# Extraer texto con Tesseract
text = pytesseract.image_to_string(image)

print(text)

‘Table 5: Case-control study of oral contraceptives and myocardial infarction [25]

as AGE Teal
25-34 35-44 45+
oc Cae Control == Case. Control
one ' 0 38 i 2
° 1 281 B 318
OR =244 OR =203
14 ' 2 35 ' Is
° 5 a 2 8
OR =253 OR =052
2+ ' u 2 8 8
° a 112 3 125
OR =7.00 OR =236
Total ' 2B 98, 10 35
° “4 64 8 coy
OR = 6.00 OR =202
(OC: oral contraceptives
(IG: cigaretes
AGE: age

Cigarettes (CIG) are sufficient to control confounding and.
that there is no misclassification or other source of bias.

We first performed a standard logistic regression analysis,
with Ml as the dependent variable and OC, AGE and CIG.
as the independent variables. As pointed out by Green-
land and Maldonado [26], there are problems identifying
the target population when using standard logistic regres-
sion analysis. Models were fit using EGRET [27|: statistical
significance of individual terms was determined using the
likelihood ratio test, and the goodness-of-fit statistic G?
was based on the deviance. On purely s

In [16]:
from PIL import Image

# Abrir la imagen original
imagen = Image.open("data/PMC1431532_00004_2861963.jpg")

# Guardar con nuevo DPI (por ejemplo, 300x300)
imagen.save("data/PMC1431532_00004_2861963_dpi.jpg", dpi=(300, 300))

In [17]:
from PIL import Image

# Cargar la imagen
image_path = "data/PMC1431532_00004_2861963_dpi.jpg"
image = Image.open(image_path)

# Extraer texto con Tesseract
text = pytesseract.image_to_string(image)

print(text)

‘We fist performed a standard logistic regression analysis,
With Mas the dependent variable and OC, AGE and CG
1s the independent variable. As pointed out by Gree.
Iand and Maldonado [26 there ae problems dentfving
the target population when using standad logistic regres
‘son analysis Models were itusing EGRET [27 staitial
‘Sgniicance of individual terns was determined using the
Tkelhood rao test, and the goodness of fi statistic G>
‘vas based on the deviance. On pute satstial grounds
the best fting model had main effects for OC, AGE and
CG, along with the interaction term AGE » CIG (C=
120, df= 8 p=-15)-The odds rao estimate forthe OC
Ml asodation was 2.2 (95% confidence intel [CI

170,468). Of note, the Mantel-Haenszel odd tio est

mate, OBjy= 282 (958 Cl 170,469), was vitally
denial 1 the logistic epession estimate. The ORs,
‘confidence interval wis based on the variance estimate
‘escribed by Robins, Breslow and Greenland [28,29] The
model wih maa effets for OC, AGE and C1G, along w

In [15]:
from PIL import Image

# Cargar la imagen
image_path = "data/PMC1431532_00004_2861963.jpg"
image = Image.open(image_path)

# Extraer texto con Tesseract
text = pytesseract.image_to_string(image, config="--dpi 0")

print(text)

‘We first performed a standard logistic regression analysis,
‘wih Mas the dependent variable and OC, AGE and CIC
{the independent variables. As pointed out by Gree
Iand and Maldonado [26 there ae problems identfving
‘the target population when using standard logistic regres
son analysis Models were itusing EGRET [27] staitial
‘Significance of individual terns was determined using the
Tikelhood rao test, and the goodness of fit statistic G*
‘vas based on the deviance. On purely statistical grounds
the best-fiting model had main effects for OC, AGE and
CG, along with the interaction term AGE » CIG (C=
120, df= 8 p=-15)-The odds ratio estimate fr the OC
NM assodation was 2.82 (95% confidence intetsal [CI

1.70468). Of note, the Mantel-Haenszel odd tio est

mate, ORjy= 282 (954 Cl: 170,469), was vitally
denial 19 the logistic egession estimate. The ORs,
‘confidence interval wis based on the valance estimate
‘escribed by Robin, Breslow and Greenland [28,29], The
model wih maa effets for OC,

In [19]:
from PIL import Image
# Cargar la imagen
image_path = "temp/scaled_document.png"
image = Image.open(image_path)

# Extraer texto con Tesseract
text = pytesseract.image_to_string(image)

print(text)
# Cargar la imagen
image_path = "temp/scaled_document_3.png"
image = Image.open(image_path)

# Extraer texto con Tesseract
text = pytesseract.image_to_string(image)

print(text)

We first performed a standard logistic regression analysis,
with ML as the dependent variable and OC, AGE and CIG
as the independent variables. As pointed out by Green-
land and Maldonado [26], there are problems identifying
the target population when using standard logistic regres-
sion analysis. Models were fit using EGRET [27]: statistical
significance of individual terms was determined using the
likelihood ratio test, and the goodness-of-fit statistic G?
was based on the deviance. On purely statistical grounds
the best-fitting model had main effects for OC, AGE and
CIG, along with the interaction term AGE « CIG (G? =
12.0, df = 8, p = .15). The odds ratio estimate for the OC-
MI association was 2.82 (95% confidence interval [Cl]:
1.70,4.68). Of note, the Mantel-Haenszel odds ratio esti-
mate, ORyy= 2.82 (95% Ch 1.70,4.69), was virtually
identical to the logistic regression estimate. The OR,
confidence interval was based on the variance estimate
described by Robins, Breslow and Gree

In [8]:
from PIL import Image

# Abrir imagen
img = Image.open("temp/scaled_document.png")

# Obtener el DPI desde los metadatos
dpi = img.info.get('dpi', 'DPI no especificado')
print(f"DPI de la imagen: {dpi}")

DPI de la imagen: DPI no especificado
